In [1]:
# Creare un ambiente virtuale
# `python3 -m venv nome_preferito`
# Attivare l'ambiente virtuale
# `source nome_preferito/bin/activate`
#  Installare i pacchetti necessari
# `pip install ipykernel`
import ace_tools_open as tools
import pandas as pd
import numpy as np


## Che cosa significa digitale?

I computer hanno una lunghezza prestabilita delle singole "parole" (word size). Si può pensare a un programma per computer come a una lunga lista di istruzioni, in cui ciascuna istruzione ha la stessa dimensione. Una CPU avrà quindi un set di istruzioni. Se ipoteticamente avessimo un computer a 3 bit, esso leggerebbe 3 bit alla volta e potrebbe avere solo 8 istruzioni ($2^3$). Supponiamo, ad esempio:  

- **000** significa "inserisci il valore successivo nel registro A"
- **001** significa "inserisci il valore successivo nel registro B"
- **010** significa "somma il registro A al registro B e memorizza il risultato in A"
- **011** significa "sottrai A da B e memorizza il risultato in A"
- **100** significa "stampa il valore del registro A sullo schermo"

Un programma potrebbe quindi apparire così:
```plaintext
000 001 001 010 100
```


Il computer legge **000**, che indica di memorizzare il valore successivo nel registro A, quindi legge **001** e lo memorizza. Poi legge **001**, che indica di memorizzare il valore successivo nel registro B (ancora **001**). Successivamente, legge **010**, che somma i valori dei registri, quindi ora il registro A contiene **001 + 001** (che è **010**). Infine, legge **100**, che stampa il valore di A sullo schermo, quindi il programma stamperà il numero **2**.

In [4]:
# In python apparirebbe così:
A = 1
B = 1
A = B + A
print(A)

2



Questo spiega anche perché i programmi compilati per un determinato chip non funzionano su un altro e perché i programmi compilati per sistemi a 64 bit non funzionano su sistemi a 32 bit. Inoltre, il motivo per cui i chip AMD e Intel possono eseguire gli stessi programmi è che entrambi seguono lo stesso standard **x64** (e **x86** per i sistemi a 32 bit), garantendo così la coerenza dei programmi. Internamente, i chip potrebbero essere diversi, ma la stessa istruzione su entrambi i chip dovrebbe produrre lo stesso risultato.
Questo è un esempio molto semplice, ma mostra come un computer possa eseguire un programma. In realtà, i computer moderni hanno una word size di 32 o 64 bit, il che significa che possono leggere 32 o 64 bit alla volta. Questo consente di avere un numero molto più grande di istruzioni e di memorizzare valori molto più grandi.  

#### Come si rappresentano le parole

Le parole `"Hello, World!"` sono memorizzate come successione di caratteri ASCII. 


| Memory Address | Contents (Hex) | ASCII Character |
|----------------|----------------|------------------|
| 0x1000         | 48             | 'H'              |
| 0x1001         | 65             | 'e'              |
| 0x1002         | 6C             | 'l'              |
| 0x1003         | 6C             | 'l'              |
| 0x1004         | 6F             | 'o'              |
| 0x1005         | 2C             | ','              |
| 0x1006         | 20             | ' '              |
| 0x1007         | 57             | 'W'              |
| 0x1008         | 6F             | 'o'              |
| 0x1009         | 72             | 'r'              |
| 0x100A         | 6C             | 'l'              |
| 0x100B         | 64             | 'd'              |
| 0x100C         | 21             | '!'              |
| 0x100D         | 0A             | Newline (`\n`)   |

Ogni carattere è memorizzato in un byte (8 bit) e la stringa è terminata da un byte nullo (`0x00`), che indica la fine della stringa. L'indirizzo di memoria indica dove inizia la stringa. Quando un programma legge una parola accede all'indirizzo di memoria in cui inizia la stringa e legge i byte successivi fino a trovare il byte nullo. Un editor di testo legge il contenuto a un certo indirizzo di memoria e lo visualizza come `"Hello, World!"`.

## I file PDF

In [ ]:
from PyPDF2 import PdfReader

# https://pypdf.readthedocs.io/
# Load the PDF
pdf_path = "/mnt/data/orario master 2024-25 8a ed.pdf"
reader = PdfReader(pdf_path)

# Extract text content and basic structural details
text_content = []
layout_structure = []

for page_num, page in enumerate(reader.pages):
    text_content.append(page.extract_text())  # Extract text content
    
    # Gathering basic layout info (text bounding boxes not available in PyPDF2)
    layout_structure.append(f"Page {page_num + 1}: {len(page.extract_text().splitlines())} lines extracted")

# Combine text content for display
full_text_content = "\n\n".join(text_content)

# Display extracted information to user
import ace_tools_open as tools; tools.display_dataframe_to_user(name="PDF Text and Layout Structure", dataframe={
    "Page Number": list(range(1, len(layout_structure) + 1)),
    "Layout Details": layout_structure
})

full_text_content[:1000]  # Displaying first 1000 characters of text content for preview

In [ ]:
import pandas as pd

# Reformat data into a DataFrame for better organization
data = {
    "Page Number": list(range(1, len(layout_structure) + 1)),
    "Layout Details": layout_structure
}
layout_df = pd.DataFrame(data)

# Displaying extracted layout details for better visualization
tools.display_dataframe_to_user(name="PDF Layout Details", dataframe=layout_df)

# Returning the first 1000 characters of the full text content for preview
full_text_content[:1000]

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.layout import LAParams, LTTextBox, LTChar
from pdfminer.pdfpage import PDFTextExtractionNotAllowed

# Function to extract font and style information
def extract_font_styles(pdf_path):
    font_data = []
    with open(pdf_path, 'rb') as file:
        parser = PDFParser(file)
        document = PDFDocument(parser)
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        resource_manager = PDFResourceManager()
        laparams = LAParams()
        for page_layout in PDFPage.create_pages(document):
            if hasattr(page_layout, 'get_text'):
                for element in page_layout:
                    if isinstance(element, LTTextBox):
                        for line in element:
                            if isinstance(line, LTChar):  # Check if character-level data exists
                                font = line.fontname
                                size = line.size
                                text = line.get_text()
                                font_data.append({"Font": font, "Size": size, "Text": text})
    return font_data

# Extract font data from the PDF
try:
    font_styles = extract_font_styles(pdf_path)
except Exception as e:
    font_styles = str(e)

font_styles[:10]  # Display a sample of the font and style data

- Esercizio 1: editare un file 

In [4]:
# url = "https://www.gutenberg.org/cache/epub/59047/pg59047.txt"
# questo è l'intero contenuto dell'Artusi.
# potete sostituirlo con qualunque altro testo vi interessi.
testo = pd.read_csv("pg59047.txt", sep="\t", header=None)
# ora il testo è contenuto in un dataframe, qualcosa di simile a un foglio di calcolo excel.
# possiamo visualizzarlo con il comando seguente, ad esempio le prime 10 righe
testo.head(10)

,0
0,The Project Gutenberg eBook of La scienza in c...
1,This ebook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...
5,at www.gutenberg.org. If you are not located i...
6,you will have to check the laws of the country...
7,before using this eBook.
8,Title: La scienza in cucina e l'arte di mangia...
9,Author: Pellegrino Artusi


In [ ]:
# oppure scorrere le righe con il comando seguente
testo.iloc[2, 0]

'most other parts of the world at no cost and with almost no restrictions'

In [ ]:
# possiamo anche caricare il file come un'unica stringa di testo
with open("pg59047.txt", encoding='utf-8') as file:
    my_data = file.read()

In [17]:
my_data[2000:4000]

"      1922\n\n\n\n\n                      Riservati i diritti d'Autore\n                           a termini di Legge\n\n\n\n\nLA STORIA DI UN LIBRO\n\nCHE RASSOMIGLIA\n\nALLA STORIA DELLA CENERENTOLA\n\n\n                               _Vedi giudizio uman come spesso erra._\n\nAvevo data l'ultima mano al mio libro _La scienza in cucina e l'Arte di\nmangiar bene_, quando capitò in Firenze il mio dotto amico Francesco\nTrevisan, professore di belle lettere al liceo Scipione Maffei di\nVerona. Appassionato cultore degli studi foscoliani, fu egli eletto\na far parte del Comitato per erigere un monumento in Santa Croce al\nCantor dei Sepolcri. In quella occasione avendo avuto il piacere di\nospitarlo in casa mia, mi parve opportuno chiedergli il suo savio\nparere intorno a quel mio culinario lavoro; ma ohimè! che, dopo averlo\nesaminato, alle mie povere fatiche di tanti anni pronunziò la brutta\nsentenza: _Questo è un libro che avrà poco esito._\n\nSgomento, ma non del tutto convinto dell

In [ ]:
# e poi esportarla in un altro file 
with open('data.txt', 'w') as file:
    file.writelines(my_data[2000:4000])

Ora aprite il file e modificatene il contenuto a vostra scelta. E non dimenticate di inserire vidoe o audio (ref https://markdown-videos.jorgenkh.no/) Tenete in considerazione le funzionalità del formato markdown:
- `#` per i titoli
- `**` per il testo in grassetto
- `*` per il testo in corsivo
- `---` per la linea orizzontale
- `![descrizione](link all'immagine)` per le immagini
- `[testo](link)` per i link
- ` ``` ` per il codice
- `>` per le citazioni
ecc.  
Ref. https://pandoc.org/MANUAL.html#pandocs-markdown  
Se non lo avete ancora fatto dovete installare pandoc sulla vostra macchina.  
Per farlo seguite le istruzioni riportate al link https://pandoc.org/installing.html

In [39]:
!pandoc data.md -o output.html -f commonmark_x+footnotes  -t html

In [ ]:
with open('output.html', 'r') as f:
    html_content = f.read()

from IPython.display import display, HTML
display(HTML(html_content))

- Esercizio 2: convertire un file in ePub

Leggere https://pandoc.org/epub.html   
Se volete potete usare il file css qui allegato `epub.css` per personalizzare il vostro file ePub.

In [ ]:
%pwd  #look at the current work dir
%cd   progit/en

In [2]:
import re
import glob

# Define the pattern to match
pattern = re.compile(r'^Insert\s*(.*)\.png\s*\n([^\n]*)$', re.MULTILINE)

# Iterate over all markdown files in subdirectories
for filepath in glob.glob('*/**/*.markdown', recursive=True):
    with open(filepath, 'r') as file:
        content = file.read()

    # Perform the substitution
    new_content = pattern.sub(r'![\2](../figures/\1-tn.png)', content)

    # Write the modified content back to the file
    with open(filepath, 'w') as file:
        file.write(new_content)

In [10]:
!pandoc -o progit.epub title.txt \
  01-introduction/01-chapter1.markdown \
  02-git-basics/01-chapter2.markdown \
  03-git-branching/01-chapter3.markdown \
  04-git-server/01-chapter4.markdown \
  05-distributed-git/01-chapter5.markdown \
  06-git-tools/01-chapter6.markdown \
  07-customizing-git/01-chapter7.markdown \
  08-git-and-other-scms/01-chapter8.markdown \
  09-git-internals/01-chapter9.markdown \
  --css epub.css \
  --epub-embed-font='fonts/ttf/*.ttf' \
  --epub-cover-image=cover.png